# 와인 리뷰데이터 인덱싱
- 벡터 DB : pinecone

In [1]:
from dotenv import load_dotenv
import os

# load_dotenv(dotenv_path="../.env")
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
HF_EMBEDDING_MODEL = "llama-text-embed-v2"
# PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_INDEX_NAME = "wine-review2"
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
# PINECONE_NAMESPACE

# 데이터 로딩

In [2]:
from langchain_community.document_loaders import CSVLoader
import os

csv_path = os.path.abspath("wine_reviews/winemag-data-130k-v2.csv")
loader = CSVLoader(csv_path, encoding="latin-1")   # 또는 "ISO-8859-1")
docs = loader.load()

# "prepare/wine_reviews/winemag-data-130k-v2.csv"
# loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
# docs = loader.load()

In [3]:
len(docs)

129971

# Embedding 모델 객체 생성
- huggingface 모델 사용 : `llama-text-embed-v2`
- gpu 버전 : pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
- pip install sentence-transformers
- pip install transformers==4.47.1 

In [5]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import time

# 1. 임베딩 모델 로드
model = SentenceTransformer(
    "nvidia/llama-nemotron-embed-1b-v2",
    trust_remote_code=True
)

In [6]:
# 모델 정보 확인
print(f"임베딩 차원: {model.get_sentence_embedding_dimension()}")  # 2048

임베딩 차원: 2048


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
import time

# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(
    model_name="nvidia/llama-nemotron-embed-1b-v2",
    model_kwargs={
        # 'device': 'cpu',
        'device': 'cuda',
        'trust_remote_code': True  
    },  # GPU 사용
    encode_kwargs={'normalize_embeddings': True}
)

# Pinecone객체, index객체 생성

In [9]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [10]:
# pinecone에 index list 가져오기
existing_indexes = pc.list_indexes()

# 이름만 추출
index_names = [index['name'] for index in existing_indexes.indexes]
# print(index_names)

# index 이름이 존재 하지 않으면 생성
if PINECONE_INDEX_NAME not in index_names:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=2048,  # 모델 차원, openapi embeding model을 사용함. 정확하게 일치
        metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully.")
else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists.")

Index 'wine-review2' already exists.


In [11]:
# split하기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [ ]:
# vector sotre에 저장(2시간정도 걸림)
from langchain_pinecone import PineconeVectorStore

BATCH_SIZE = 100  # 한 번에 처리할 문서 수(최대 vector 수 1000개, 2MB 이내)

for i in range(0, len(chunks), BATCH_SIZE):
    batch_docs = chunks[i:i+BATCH_SIZE]
    
    # 첫 번째 배치로 벡터 스토어 생성
    if i == 0:
        vector_store = PineconeVectorStore.from_documents(
            batch_docs,            # BATCH_SIZE 수 만큼의 chunk
            embedding=embeddings,  # 임베딩 벡터로 변환
            index_name=PINECONE_INDEX_NAME,   # index 이름
            namespace=PINECONE_NAMESPACE      
        )

    # 이후 배치는 생성한 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
    else:
        vector_store.add_documents(batch_docs)    
    
    print(f"배치 {i//BATCH_SIZE + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 100개 문서 업로드
